In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras 
from matplotlib import pyplot as plt
from keras.utils import load_img, img_to_array,normalize
import os
from PIL import Image
from keras.optimizers import Adam
from keras.losses import MeanAbsoluteError 



2024-06-18 07:55:13.222993: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-18 07:55:13.223179: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-18 07:55:13.398394: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
def load_images_from_folder(folder_path, target_size=(600,400)):
    images = []
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        if os.path.isfile(img_path):
            try:
                img = Image.open(img_path)
                img = img.resize(target_size) 
                img_array = np.array(img)
                if img_array.shape[-1] == 3:  
                    img_array = img_array / 255.0  
                    images.append(img_array)
            except Exception as e:
                print(f"Error loading image {img_path}: {e}")
    return np.array(images)

In [3]:
low_array = load_images_from_folder('/kaggle/input/low-light-denoising/Train/low')
high_array = load_images_from_folder('/kaggle/input/low-light-denoising/Train/high')

In [4]:
train_low=low_array[:int(0.9*low_array.shape[0])]
test_low=low_array[int(0.9*low_array.shape[0]):]

train_high=high_array[:int(0.9*low_array.shape[0])]
test_high=high_array[int(0.9*low_array.shape[0]):]

In [20]:
model=keras.Sequential([
    keras.layers.Convolution2D(6,3,padding='same',strides=1,activation="relu",input_shape=(400,600,3)),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Convolution2D(12,3,padding='same',activation="relu",strides=1),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Convolution2D(24,3,padding='same',activation="relu",strides=1),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Convolution2D(48,3,padding='same',activation="relu",strides=1),
    keras.layers.UpSampling2D((2,2)),
    keras.layers.Convolution2D(24,3,padding='same',activation="relu",strides=1),
    keras.layers.UpSampling2D((2,2)),
    keras.layers.Convolution2D(12,3,padding='same',activation="relu",strides=1),
    keras.layers.UpSampling2D((2,2)),
    keras.layers.Convolution2D(6,3,padding='same',activation="relu",strides=1),
    keras.layers.Convolution2D(3,3,padding='same',strides=1,activation='sigmoid'),
])

print(model.summary())

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_16 (Conv2D)              │ (None, 400, 600, 6)    │           168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 200, 300, 6)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 200, 300, 12)   │           660 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 100, 150, 12)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, 100, 150, 24)   │         2,616 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 50, 75, 24)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 50, 75, 48)     │        10,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_6 (UpSampling2D)  │ (None, 100, 150, 48)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 100, 150, 24)   │        10,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_7 (UpSampling2D)  │ (None, 200, 300, 24)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_21 (Conv2D)              │ (None, 200, 300, 12)   │         2,604 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_8 (UpSampling2D)  │ (None, 400, 600, 12)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_22 (Conv2D)              │ (None, 400, 600, 6)    │           654 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_23 (Conv2D)              │ (None, 400, 600, 3)    │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 27,675 (108.11 KB)

 Trainable params: 27,675 (108.11 KB)

 Non-trainable params: 0 (0.00 B)

None


In [21]:
def calculate_psnr(data,test_data):
            output=model(data)
            output_image= tf.cast(output,tf.float32)
            test_data =tf.cast(test_data,tf.float32)
            squared_diff = tf.square(output_image - test_data)
            mse = tf.reduce_mean(squared_diff)
            max_pixel = 1.0
            psnr = (20 * tf.math.log(max_pixel) / tf.math.log(10.0)) - (10 * tf.math.log(mse) / tf.math.log(10.0))
            return psnr.numpy()

In [22]:
model.compile(optimizer=Adam(learning_rate=0.01),loss=MeanAbsoluteError(), metrics=['accuracy'])
model.fit(train_low,train_high,batch_size=32,epochs=10)

Epoch 1/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 123s 8s/step - accuracy: 0.3082 - loss: 0.1831
Epoch 2/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 120s 9s/step - accuracy: 0.4976 - loss: 0.1560
Epoch 3/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 132s 9s/step - accuracy: 0.5545 - loss: 0.1583
Epoch 4/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 126s 9s/step - accuracy: 0.5404 - loss: 0.1517
Epoch 5/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 120s 9s/step - accuracy: 0.5557 - loss: 0.1463
Epoch 6/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 142s 9s/step - accuracy: 0.5278 - loss: 0.1542
Epoch 7/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 123s 9s/step - accuracy: 0.5432 - loss: 0.1515
Epoch 8/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 137s 10s/step - accuracy: 0.5389 - loss: 0.1401
Epoch 9/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 121s 9s/step - accuracy: 0.5461 - loss: 0.1444
Epoch 10/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 120s 9s/step - accuracy: 0.5485 - loss: 0.1415


In [23]:
calculate_psnr(test_low,test_high)

15.496921